In [3]:
%matplotlib inline

import shutil
import os
from joblib import dump, load
import urllib3
import certifi
from Bio import SeqIO
import Bio
from glob import glob
import json
from IPython import display
import pandas as pd
import numpy as np
import networkx as nx
from collections import OrderedDict

import matplotlib.pyplot as plt
import seaborn as sns
import re

![HIV genome](https://upload.wikimedia.org/wikipedia/commons/c/c6/HIV-genome.png)

In [ ]:
patients = ["p{}".format(i) for i in range(1,12)]
hiv_regions = ["V3", "PR", "psi", "vpr", "vpu", "p1", "p2", "p6", "p7", "p15", "p17", "RRE"]

![HIV genome details](https://res.mdpi.com/viruses/viruses-08-00248/article_deploy/html/images/viruses-08-00248-g002.png)

In [ ]:
def download_hivevo_haplotype(patient, hiv_region, folder):
    
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',
                               ca_certs=certifi.where())
    
    api = "https://hiv.biozentrum.unibas.ch/api/data/haplotypes/"
    
    url = "/".join((api, patient, hiv_region))

    if not os.path.isdir(folder):
        os.mkdir(folder)

    file_name = folder + "_".join(("hivevo", patient, region)) + ".fasta"

    with http.request('GET', url, preload_content=False) as res, open(file_name, 'wb') as out_file:

        shutil.copyfileobj(res, out_file)
        
folder = "data/"
for patient in patients:
    for region in hiv_regions:
        download_hivevo_haplotype(patient, region, folder)

In [ ]:
def download_hivevo_references(folder):
    global patients    
    
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',
                               ca_certs=certifi.where())
    
    if not os.path.isdir(folder):
        os.mkdir(folder)
    
    for patient in patients:
        api = "https://hiv.biozentrum.unibas.ch/api/data/referenceSequence"
        url = "/".join((api, patient))
        file_name = folder + "_".join(("hivevo", "reference", patient)) + ".fasta"
        
        with http.request('GET', url, preload_content=False) as res, open(file_name, 'wb') as out_file:
            shutil.copyfileobj(res, out_file)

folder = "data/references/"
download_hivevo_references(folder)

In [ ]:
def extracting_region_from_reference(region, reference_path, folder):
    
    if not os.path.isdir(folder+region):
        os.mkdir(folder+region)
    
    with open(reference_path) as f:
        reference_info = json.load(f)
    for reg in reference_info['features']:
        if reg['name'] == region:
            loc = reg['location'][0]
            break
        else:
            continue
    patient = re.search(r'p[\d]*', reference_path)[0]
    #print(patient)
    sequence = reference_info['seq'][loc[0]:loc[1]]
    #print(sequence)
    res = r'/'+re.search(r'[\w]*\.fasta', reference_path)[0].replace(patient, "_".join((patient, region)))
    
    with open(folder+region+res, 'w') as write_file:
        write_file.write('>'+reference_info['description'].replace('genomewide', 'region='+region)+'\n')
        write_file.write(sequence)

folder = 'data/references/'
region = 'V3'
ref_path = 'data/references/hivevo_reference_p4.fasta'
extracting_region_from_reference(region, ref_path, folder)

In [ ]:
path = 'sobaka'
"/".join((folder, path))

In [ ]:
reference_path = 'data/references/hivevo_reference_p4.fasta'
res = re.search(r'[\w]*\.fasta', reference_path)[0]
res

In [ ]:
with open('data/references/hivevo_reference_p4.fasta') as file:
    json_ref = json.load(file)
#json_ref

In [ ]:
haplotype_lst = glob('data/*V3.fasta')
reference_lst = glob('data/references/*.fasta')
#reference_lst

In [ ]:
tdf = pd.DataFrame()
for path in haplotype_lst:
    with open(path, "r") as f:
        records = json.load(f)
    df = pd.DataFrame(records)
    df = df.drop("description", axis=1)
    df["patient"] = path.split("_")[1]
    tdf = pd.concat([tdf, df])

In [ ]:
tdf.groupby("patient").agg({"days since infection" : ["min", np.median, "max", pd.Series.nunique],
                            "patient": ["size"]}).sort_index()

In [ ]:
tdf.columns = ["days", "frequency", "name", "sequence", "patient"]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 6))
sns.stripplot(y="patient", x="days", data=tdf, )

I will remove p10, because there are too few time points and p7, because it's too different from other samples.

In [ ]:
tdf = tdf[~tdf.patient.isin(["p7", "p10"])]
tdf.shape

In [ ]:
tdf.columns = ["days", "frequency", "name", "sequence", "patient"]

In [ ]:
_, bin_edges = np.histogram(tdf.days.values, 5)
bin_edges[0] -= 1
bin_edges[-1] += 1
tdf["bins"] = np.digitize(tdf.days.values, bin_edges)
tdf.head()

In [ ]:
tdf.loc[tdf['patient'] == 'p3']

In [ ]:
tdf.groupby(["patient", "bins"]).size().unstack(fill_value=0)

In [ ]:
tdf = tdf.drop("bins", axis=1)
# Will analyse everyone separately

In [ ]:
re.search(r'\b[\w\/]+p4[\w\.]+\b', ' '.join(file_lst))[0]

In [ ]:
' '.join(file_lst)

In [ ]:
tdf[tdf.patient == "p4"].groupby('days').size()

In [ ]:
tdf = tdf.reset_index(drop=True)
tdf.loc[1, "sequence"]

In [ ]:
path_json = 'data/hivevo_p4_V3.fasta'

with open(path_json) as f:
    json_file = json.load(f)
json_file[0]

### Major haplotypes stuff

In [ ]:
def finding_major_freq(json_file):
    dict_days = {}
    for obj in json_file:
        day = obj['days since infection']
        if day in dict_days:
            if dict_days[day] < obj['frequency [%]']:
                dict_days[day] = obj['frequency [%]']
        else:
            dict_days[day] = obj['frequency [%]']
    return dict_days

finding_major_freq(json_file)

In [25]:
def getting_major_haplotypes(folder, region, patient):
    '''
    finding max freq haplotype for each day we know
    '''
    if not os.path.isdir(folder+'major'):
        os.mkdir(folder+'major')
    
    path_json = folder + "/" + "_".join(('hivevo', patient, region)) + r'.fasta'
    path = path_json.replace(folder, folder+'/major')
    
    with open(path, "w") as fasta_file, open(path_json) as json_f:
        json_file = json.load(json_f)
        dict_days = finding_major_freq(json_file)
        for obj in json_file:
            if obj['frequency [%]'] == dict_days[obj['days since infection']]:
                line_1 = '>' + obj['name'] + '\n'
                line_2 = obj['sequence'] + '\n'
                lines = [line_1, line_2]
                fasta_file.writelines(lines)
            else:
                continue
getting_major_haplotypes('data/','V3','p4')

### Making json to fasta

In [ ]:
def json2fasta(folder, path_json):
    
    if not os.path.isdir(folder+'/fasta'):
        os.mkdir(folder+'/fasta')
        
    with open(path_json) as f:
        json_file = json.load(f)
    
    path = path_json.replace('data/', 'data/fasta/')   
    with open(path, "w") as fasta_file:
        i = 0
        for obj in json_file:
            line_1 = '>' + obj['name'] + str(i) + '\n'
            line_2 = obj['sequence'] + '\n'
            lines = [line_1, line_2]
            fasta_file.writelines(lines)
            i += 1

In [ ]:
!cat data/references/V3/hivevo_reference_p4_V3.fasta

In [26]:
def add_ref2fasta(path_fasta, path_ref_region):
    with open(path_fasta, 'a') as fasta, open(path_ref_region) as ref:
        for line in ref:
            #print(line)
            fasta.write(line)
            
add_ref2fasta('data/major/hivevo_p4_V3.fasta', 'data/references/V3/hivevo_reference_p4_V3.fasta')

### Alignments

In [ ]:
from Bio.Align.Applications import ClustalwCommandline
clustalw_cline = ClustalwCommandline("clustalw",  align = 'True', infile="data/fasta/hivevo_p4_V3.fasta", output = 'FASTA', outfile = 'data/clustal_output/hivevo_p4_V3.fasta', type = 'DNA')
stdout, stderr = clustalw_cline()
#print(clustalw_cline)

In [ ]:
# SHOULD BE USED https://biopython.org/DIST/docs/api/Bio.Phylo.Applications._Fasttree-module.html

!fasttree -nt -gtr < data/clustal_output/hivevo_p4_V3.fasta > data/trees/hivevo_p4_V3.nwk

In [ ]:
from Bio import Phylo

tree = Phylo.read('data/trees/hivevo_p4_V3.nwk', 'newick')
#print(tree)

In [ ]:
graph = Phylo.to_networkx(tree)

In [ ]:
tree.total_branch_length()

In [ ]:
def clade_names_fix(tree):
    for idx, clade in enumerate(tree.find_clades()):
        if not clade.name:
            clade.name=str(idx)

Tree = Phylo.read('data/trees/hivevo_p4_V3.nwk', 'newick')
clade_names_fix(Tree)
G = Phylo.to_networkx(Tree)
nx.write_graphml(G, 'data/graphml/hivevo_p4_V3.graphml')

In [ ]:
G = nx.read_graphml('data/graphml/hivevo_p4_V3.graphml')

source = '0'
dist_dict = nx.shortest_path_length(G, '0')

import operator 
target = max(dist_dict.items(), key=operator.itemgetter(1))[0]

graph_path = nx.shortest_simple_paths(G, source, target)

In [ ]:
lst_path = list(graph_path)
lst_path

### Another approach

In [1]:
from Bio.Align.Applications import ClustalwCommandline
clustalw_cline = ClustalwCommandline("clustalw",  align = 'True', tree = 'True', infile="data/fasta/hivevo_p4_V3.fasta", output = 'FASTA', outfile = 'data/clustal_output/hivevo_p4_V3.fasta', type = 'DNA')
stdout, stderr = clustalw_cline()
#print(clustalw_cline)

In [2]:
#get rid of .dnd file

def getrid_dnd(patient_str):
    '''Void func converting .dnd file to .nwk'''
    tmp = '_v0'
    while os.path.exists('data/trees/'+patient_str+tmp+'.nwk'):
        print(tmp)
        tmp = tmp[0:2] + str(int(tmp[-1]) + 1)
    with open('data/fasta/'+patient_str+'.dnd') as dnd, open('data/trees/'+patient_str+tmp+'.nwk','w') as nwk:
        row = ''
        for line in dnd:
            row += line.rstrip()
        nwk.write(row)

patient_str = 'hivevo_p4_V3'
getrid_dnd(patient_str)

NameError: name 'os' is not defined

In [ ]:
Tree = Phylo.read('data/trees/hivevo_p4_V3_v0.nwk', 'newick')
#print(Tree)

In [ ]:
clade_names_fix(Tree)
G = Phylo.to_networkx(Tree)
nx.write_graphml(G, 'data/graphml/hivevo_p4_V3.graphml')

In [ ]:
lst_nodes = list(G.nodes)
lst_edges = list(G.edges)

In [ ]:
G = nx.read_graphml('data/graphml/hivevo_p4_V3.graphml')

source = lst_nodes[0]
dist_dict = nx.shortest_path_length(G, source)

In [ ]:
dist_dict

In [ ]:
target = max(dist_dict.items(), key=operator.itemgetter(1))[0]

graph_path = nx.shortest_simple_paths(G, source, target)

In [ ]:
lst_path = list(graph_path)
lst_path

### using Muscle